In [1]:
import numpy as np
import tensorflow as tf
from curriculum_learning.models.classifier_model import ClassifierModel
from curriculum_learning import utils
import yaml
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
with open("models_hyperparameters.yaml", "r") as stream:
    models_hyperparameters = yaml.safe_load(stream)
    
with open("config_tests.yaml", "r") as stream:
    config_tests = yaml.safe_load(stream)
    
N_EPOCHS = 50
N_TRIALS = 10
BATCH_SIZE = 512

CONFIG = config_tests["proba_best"]

loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [3]:
ds_1 = tfds.load("stl10", split="train", as_supervised=True, shuffle_files=False)
ds_2 = tfds.load("stl10", split="test", as_supervised=True, shuffle_files=False)

x = []
y = []
for x_, y_ in ds_1.as_numpy_iterator():
    x.append(x_)
    y.append(y_)
for x_, y_ in ds_2.as_numpy_iterator():
    x.append(x_)
    y.append(y_)
    
x = np.array(x, dtype=np.float32) / 255
y = np.array(y, dtype=np.float32)

2024-04-28 08:07:02.218319: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-28 08:07:02.970800: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=42, stratify=y)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42, stratify=y_test)

n_classes = len(np.unique(y))
train_size = x_train.shape[0]
train_size, len(x_val), len(x_test)

(7800, 2600, 2600)

In [5]:
x_train_sorted = x_train[np.argsort(y_train)]
y_train_sorted = y_train[np.argsort(y_train)]
_, counts = np.unique(y_train_sorted, return_counts=True)

In [11]:
model = ClassifierModel(output_shape=n_classes, **models_hyperparameters["test_model"])
model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
model(x_train[0:1])
# model.save_weights("../models/default_model.weights.h5")
model.load_weights("../models/default_model.weights.h5")
model_weights = model.get_weights()

In [22]:
model_scores = []
verbose = 1
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=5, start_from_epoch=0)

for _ in tqdm(range(N_TRIALS)):
    model.set_weights(model_weights)
    
    for j in range(len(model.layers[:-1])):
        model.layers[j].layers[-1].rate = 0.25
    
    for i in range(N_EPOCHS):
        n_samples = int(np.tanh(2 * (i + 1) / N_EPOCHS) * train_size)
        
        samples_proba = utils.calculate_proba(
            model, x_train_sorted, y_train_sorted, counts
        )
        
        samples_ids = utils.chose_samples(n_samples, samples_proba, CONFIG["order_type"])
        model.fit(
            x_train_sorted[samples_ids],
            y_train_sorted[samples_ids],
            # validation_data=(x_val, y_val),
            epochs=1,
            batch_size=BATCH_SIZE,
            verbose=verbose,
        )
        
        for j in range(len(model.layers[:-1])):
            model.layers[j].layers[-1].rate -= 0.003
    
    model.fit(
        x_train, y_train, validation_data=(x_val, y_val), epochs=500, batch_size=BATCH_SIZE, 
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=5, start_from_epoch=10)], 
        verbose=verbose
    )
            
    _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=verbose)
    model_scores.append(accuracy)
    print("Mean:", np.mean(model_scores), " Median: ", np.median(model_scores))

  0%|          | 0/10 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 526ms/step - accuracy: 0.1222 - loss: 2.5450
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 384ms/step - accuracy: 0.1121 - loss: 2.5220
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 431ms/step - accuracy: 0.1627 - loss: 2.3599
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 263ms/step - accuracy: 0.1762 - loss: 2.2394
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 156ms/step - accuracy: 0.2508 - loss: 2.0938
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 147ms/step - accuracy: 0.2283 - loss: 2.0817
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - accuracy: 0.2467 - loss: 2.0499
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step - accuracy: 0.2576 - loss: 2.0257
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 123ms/step - accuracy: 0.2573 - loss: 1.9943
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 129ms/step - accuracy: 0.2922 - loss: 1.9198
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step - accuracy: 0.3054 - loss: 1.8878
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step - accuracy: 0.3051 - loss: 1.8517
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - accuracy: 0.3351 - loss: 1.8026
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step - accuracy: 0

 10%|█         | 1/10 [02:21<21:10, 141.12s/it]

Mean: 0.5776923298835754  Median:  0.5776923298835754
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 470ms/step - accuracy: 0.0932 - loss: 2.5338
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 375ms/step - accuracy: 0.1160 - loss: 2.4757
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 516ms/step - accuracy: 0.1666 - loss: 2.3423
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 211ms/step - accuracy: 0.1994 - loss: 2.2438
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 149ms/step - accuracy: 0.2223 - loss: 2.2173
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step - accuracy: 0.2519 - loss: 2.0835
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 153ms/step - accuracy: 0.2282 - loss: 2.1029
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step - accuracy: 0.2367 - loss: 2.0699
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 121ms/step - accuracy: 0.2683 - loss: 2.0057
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step - accuracy: 0.2628 - loss: 1.9853
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 102ms/step - accuracy: 0.2814 - loss: 1.9492
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 99ms/step - accuracy: 0.3118 - loss: 1.9001
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 102ms/step - accuracy: 0.3050 - loss: 1.881

 20%|██        | 2/10 [04:35<18:16, 137.12s/it]

...

Mean: 0.5587179660797119  Median:  0.556538462638855
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 362ms/step - accuracy: 0.1158 - loss: 2.5523
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 392ms/step - accuracy: 0.0982 - loss: 2.5356
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 417ms/step - accuracy: 0.1482 - loss: 2.3967
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step - accuracy: 0.1604 - loss: 2.3254
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 141ms/step - accuracy: 0.2155 - loss: 2.1978
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 150ms/step - accuracy: 0.2204 - loss: 2.1478
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step - accuracy: 0.2000 - loss: 2.2096
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 143ms/step - accuracy: 0.2142 - loss: 2.1310
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step - accuracy: 0.2584 - loss: 2.0187
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 126ms/step - accuracy: 0.2465 - loss: 1.9993
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 123ms/step - accuracy: 0.2936 - loss: 1.9453
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 100ms/step - accuracy: 0.2984 - loss: 1.9044
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - accuracy: 0.2700 - loss: 1.9103

 40%|████      | 4/10 [09:13<13:53, 138.85s/it]

Mean: 0.5515384674072266  Median:  0.5492307841777802
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 435ms/step - accuracy: 0.1318 - loss: 2.4557
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 375ms/step - accuracy: 0.1171 - loss: 2.5445
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 424ms/step - accuracy: 0.1618 - loss: 2.3835
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step - accuracy: 0.1831 - loss: 2.2896
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 157ms/step - accuracy: 0.2062 - loss: 2.1340
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 157ms/step - accuracy: 0.2127 - loss: 2.1548
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 145ms/step - accuracy: 0.2150 - loss: 2.1205
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 144ms/step - accuracy: 0.2601 - loss: 2.0304
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step - accuracy: 0.2623 - loss: 1.9902
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - accuracy: 0.2757 - loss: 1.9581
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 104ms/step - accuracy: 0.2848 - loss: 1.9226
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 102ms/step - accuracy: 0.3342 - loss: 1.8359


 40%|████      | 4/10 [09:36<14:24, 144.05s/it]


KeyboardInterrupt: 

In [17]:
model_scores

[0.5615384578704834,
 0.5284615159034729,
 0.5350000262260437,
 0.5380769371986389]

In [8]:
model.layers[1].layers[-1].rate

0.1359999999999999

In [23]:
model_scores_random = []
verbose = 0

for _ in tqdm(range(N_TRIALS)):
    model.set_weights(model_weights)

    for j in range(len(model.layers[:-1])):
        model.layers[j].layers[-1].rate = 0.1

    model.fit(
        x_train, y_train, validation_data=(x_val, y_val), epochs=500, batch_size=BATCH_SIZE, verbose=verbose, 
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=5, start_from_epoch=35)]
    )

    _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=verbose)
    model_scores_random.append(accuracy)
    print("Mean:", np.mean(model_scores_random), " Median: ", np.median(model_scores_random))

  0%|          | 0/10 [00:19<?, ?it/s]


KeyboardInterrupt: 

In [19]:
model_scores_random

[0.574999988079071, 0.5753846168518066, 0.5699999928474426]

In [20]:
np.mean(model_scores), np.mean(model_scores_random)

(0.5407692342996597, 0.5734615325927734)

In [34]:
np.mean(sorted(model_scores)[5:]), np.mean(sorted(model_scores_random)[5:])

/Users/dbugajny/PycharmProjects/CurriculumLearning/venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dbugajny/PycharmProjects/CurriculumLearning/venv/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


(nan, nan)

In [36]:
import scipy.stats
scipy.stats.ttest_ind(model_scores, model_scores_random)

TtestResult(statistic=3.476015034822166, pvalue=0.008368460131238117, df=8.0)